In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    TREC_EVAL="/content/drive/MyDrive/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"
else:
    WORKING_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/unicamp_IR/"
    TREC_EVAL="/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"

In [4]:
import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [60]:
ALL_EVALUATIONS="tests/evaluation_17_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240128.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

E5_RUN="tests/e5_docs_000_999_50_sampled_queries_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_999_50_sampled_queries_run.txt"
SPLADE_MT5_RUN="tests/splade_mt5_docs_000_099_50_sampled_queries_run.txt"

BM25_1M_RUN="tests/bm25_1M_docs_000_999_sampled_50_queries_run.txt"
BM25_1M_MASSIVEWEB_RUN="tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

COLBERTX_1M_MASSIVEWEB_RUN="tests/colbertx_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

E5_COLBERTX_RRF_RUN="tests/colbertx_e5_RRF_000_999_sampled_50_queries_run.txt"
E5_SPLADE_RRF_RUN="tests/splade_e5_fixed_RRF_000_999_sampled_50_queries_run.txt"

ADAV2_RUN="tests/adav2_1M_docs_000_999_sampled_50_queries_run.txt"

BM25_MT5_1M_MASSIVEWEB_RUN="tests/bm25_mt5_1M_docs_fixed_000_099_sampled_50_queries_run.txt"

In [56]:
COLBERTX_1M_MASSIVEWEB="tests/colbertx_massiveweb_1M_docs_000_999_sampled_50_queries.tsv"

BM25_E5_1M_MASSIVEWEB="tests/bm25_e5_1M_docs_fixed_000_999_sampled_50_queries.tsv"

In [8]:
DATASET_1M_FIELDS=['bm25_1M_massiveweb', 'colbertx_1M_massiveweb', 'adav2', 'bm25_mT5_1M_massiveweb_fix', 'bm25_e5_1M_massiveweb']

In [9]:
def prepare_run_format(results_filename, add_index=True, score_field='score', invert_score=False, retriever_name="retriever"):

    results_df = pd.read_csv(results_filename, sep='\t')

    display(results_df.head())

    results_grouped_df = results_df.groupby("query_id")['passage_id'].count()

    how_many_passages_per_question = results_grouped_df.iloc[0]

    if invert_score:
        print("Inverting score values in '{}' field...".format(score_field))

        results_df['score_inv'] = 1 - results_df[score_field]
        score_field = 'score_inv'

    run_df = results_df.sort_values(["query_id", score_field], ascending=[True, False])

    run_df['Q0'] = "Q0"
    run_df['comment'] = retriever_name

    if add_index:
        run_df['index'] = np.tile(list(range(1, how_many_passages_per_question + 1)), results_grouped_df.shape[0])

    output_filename = "{}_run.txt".format(os.path.splitext(results_filename)[0])

    print("output_filename={}".format(output_filename))

    run_df[['query_id', 'Q0', "passage_id", "index", score_field, "comment"]].to_csv(output_filename, header=None, index=False, sep=" ")

    return output_filename, run_df

In [61]:
retrievers_results = []

In [11]:
retrievers_1M_results = []

### Format GPT-4 evaluations to qrels format

In [12]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))
qrels_1M_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_1M_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [13]:
if os.path.exists(qrels_filename) == False:
    eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

    eval_df

    eval_df.drop_duplicates("query_id")[['query_id', 'query']]

    #### Check scores distribution on the current qrels

    np.unique(eval_df['score'].to_numpy(), return_counts=True)

    for group_name, group_df in eval_df.groupby('query_id'):
        print("\nquery_id={}".format(group_name))

        print(np.unique(group_df['score'].to_numpy(), return_counts=True))

    eval_df.groupby('query_id').count()

    eval_df.groupby('query_id').count()['query'].describe()

    #### Prepare the qrels format for trec_eval tools

    eval_df['0'] = '0'

    print("\n\n")
    print(qrels_filename)
    print(qrels_1M_filename)

    eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")


    #### Select 1M dataset evaluations

    eval_df[eval_df[DATASET_1M_FIELDS].sum(axis=1) > 0].sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_1M_filename, header=None, index=False, sep=" ")

else:
    print("QRELS files ({}) already created...".format(qrels_filename))


query_id=1
(array([0, 1, 2, 3]), array([21, 47, 18, 10]))

query_id=2
(array([0, 1, 2, 3]), array([111,   2,   1,   2]))

query_id=9
(array([0, 1, 2, 3]), array([72, 10, 13,  5]))

query_id=11
(array([0, 1, 2, 3]), array([56, 18,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([42, 16, 23,  4]))

query_id=15
(array([0, 1, 2, 3]), array([35, 22,  4, 17]))

query_id=17
(array([0, 1, 2, 3]), array([30, 10,  2, 39]))

query_id=20
(array([0, 1, 2, 3]), array([18, 67, 27,  7]))

query_id=21
(array([0, 1, 2, 3]), array([56, 24, 10, 21]))

query_id=22
(array([0, 1, 2, 3]), array([60, 16,  2,  7]))

query_id=24
(array([0, 1, 2, 3]), array([67, 33, 11,  2]))

query_id=26
(array([0, 1, 2, 3]), array([14, 19, 16, 33]))

query_id=28
(array([0, 1, 2, 3]), array([61, 22, 13,  4]))

query_id=47
(array([0, 1, 2, 3]), array([57, 33, 33,  1]))

query_id=49
(array([0, 1, 2, 3]), array([32, 30, 24,  2]))

query_id=51
(array([0, 1, 2, 3]), array([41, 23, 13,  1]))

query_id=54
(array([0, 1, 2, 3]), arra

## 10M dataset

### Compute BM25 nDCG@10

In [62]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [63]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [64]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4487'}]

### Compute BM25 + E5 nDCG@10

In [65]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [66]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [67]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [68]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [69]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [70]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [71]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [72]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [73]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [74]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [75]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [76]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE + mT5 nDCG@10

In [77]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_MT5_RUN}

In [78]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + ColBERT-X RRF nDCG@10

In [79]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_COLBERTX_RRF_RUN}

In [80]:
retrievers_results.append({'retriever':'E5 + ColBERT-X mMARCO_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + SPLADE RRF nDCG@10

In [81]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_SPLADE_RRF_RUN}

In [82]:
retrievers_results.append({'retriever':'E5 + SPLADEv2_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [83]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M massiveweb filtered nDCG@10

In [84]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X 1M massiveweb filtered nDCG@10

In [57]:
colbertx_filename, _ = prepare_run_format(COLBERTX_1M_MASSIVEWEB_RUN, retriever_name="ColBERT-x_pt-BR")

,query_id,1M_dataset_doc_index,index,score,passage_id,passage,Q0,comment,query
0,1,83086,1,27.924545,clueweb22-pt0000-17-01771_1,"O Brasil é um país conhecido mundialmente por suas belezas naturais, incluindo a sua fauna que contém diversas espécies de animais exóticos como a arara-azul, o mico-leão-dourado e a onça-pintada. Há ... Redação completa Enviada em: 25/04/2017 A vasta natureza brasileira, exaltada desde o século dezesseis na carta de Pero Vaz de Caminha, continua sendo um dos maiores patrimônios nacionais, mas é também o maior alvo do contrabando e das dest... Redação completa Enviada em: 26/04/2017 Em vários locais do mundo, principalmente no Brasil, agrega inúmeras espécies de animais em ecossistemas e climas distintos. A essência atualizante da sociedade traz como característica admirável, a c... Redação completa Enviada em: 26/04/2017 Nos primórdios do século XVI,os lusitanos desembarcaram nas terras brasileiras já com o ideal de exploração.Nesse contexto,até mesmo os animais silvestres eram incorporados nas práticas mercantilistas... Redação completa Enviada em: 26/04/2017 Para Paul Atson, co-fundador do Greenspace, \""a inteligência é a habilidade de se viver em harmonia com o meio ambiente\"". Desse modo, percebe-se que essa harmonia é rompida no Brasil com o tráfico de a... Redação completa Enviada em: 27/04/2017 Tráfico de animais silvestres: um problema secular",Q0,ColBERT-X_pt-BR,Qual a maior característica da fauna brasileira?
1,1,102524,2,27.615765,clueweb22-pt0000-21-03309_40,"São surpreendentemente uniformes, pois suas características mais óbvias são as penas e o bico córneo. São chamadas de endotérmicas , porque produzem seu próprio calor e de homoeotérmicas , porque podem manter a temperatura de seus corpos razoavelmente alta e constante. Isto não significa que a temperatura do corpo de uma ave não varie, pode haver oscilação diária de vários graus. São tetrápodas, com o par anterior transformado em asas e o posterior adaptado para empoleirar, andar ou nadar. A capacidade de voar permite às aves ocupar alguns hábitats impossíveis para outros animais. Além da quantidade, a avifauna do Brasil reúne inúmeros superlativos quanto à qualidade. Vive aqui, uma das maiores aves do mundo, a ema, ao lado das aves de menor porte, os beija-flores. Encontram-se os voadores de maior porte da Terra: o albatroz e o condor, ambos de ocorrência apenas ocasional. O gavião-real, residente no Brasil, é a ave de rapina mais possante do mundo. Ocorrem aqui as aves de vôo mais veloz: falcões e andorinhões.",Q0,ColBERT-X_pt-BR,Qual a maior característica da fauna brasileira?
2,1,782171,3,27.508310,clueweb22-pt0001-63-03254_3,"No Brasil, tratando-se de árvores nativas, é muito comum encontrar a mesma espécie, em várias regiões do país, como por exemplo o ipê, a canafístula, o manacá, que se desenvolvem bem em climas e temperaturas diversas. Há ainda, aquelas que são símbolos de representação, como por exemplo a Araucária no estado do Paraná ou o próprio Pau Brasil, árvore nativa das florestas tropicais da costa brasileira que deu origem ao nome do nosso país. Para conhecer a lista de espécies nativas por estado, acesse este link. Quanto às árvores exóticas, listamos as principais características das espécies mais populares e presentes no Brasil: Khaya grandifoliola (ivorensis/grandifoliola) – Mogno Africano O mogno africano é originário das regiões tropicais de baixa altitude da África Ocidental, abrange países como Costa do Marfim, Gana, Togo, Benin, Nigéria e Angola. Sua árvore pode atingir de 40 a 60 metros de altura e diâmetro de 2,10 m. Se desenvolve melhor em áreas mais quentes e não resiste à regiões com presença de geadas. Por ser considerada nobre, é uma das espécies madeireiras mais importantes para plantios comerciais, combinando crescimento rápido, boa qualidade da madeira e excelente cotação no mercado internacional.",Q0,ColBERT-X_pt-BR,Qual a maior característica da fauna brasileira?
3,1,961539,4,27.382154,cluew

output_filename=tests/colbertx_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt


In [85]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADAv2 1M massiveweb filtered nDCG@10

In [86]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {ADAV2_RUN}

retrievers_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 1M massiveweb filtered nDCG@10

In [87]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25+mT5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 + E5 1M massiveweb filtered run and compute nDCG@10

In [88]:
bm25_e5_filename, _ = prepare_run_format(BM25_E5_1M_MASSIVEWEB, retriever_name="BM25+E5")

,query,colbertx_id,score,query_id,passage_id,passage,n_tokens
0,Qual a maior característica da fauna brasileira?,4875435,84.761627,1,clueweb22-pt0001-01-18874_0,"Conheça 11 lindos animais típicos da fauna brasileira / Incrível Conheça 11 lindos animais típicos da fauna brasileira 4--4 Compartilhe no Facebook Compartilhe no Facebook Compartilhe no Pinterest Que o Brasil é gigante, todos nós sabemos e até já demos aqui exemplos chocantes sobre isso. Mas um outro exemplo dessa grandiosidade é quando sabemos que o país abriga 17% das espécies de aves e 10% das de anfíbios e mamíferos de todo o mundo. Neste post, o Incrível.club vai te mostrar uma lista de animais típicos do país. Mico-leão dourado © Nadine Doerle / Pixabay Este macaco “ruivo” ficou famoso ao tornar-se um símbolo de espécies brasileiras em extinção. O cuidado para a preservação da espécie, que é nativa da Mata Atlântica e aparece na nota de 20 reais, começou nos anos 1970 e se intensificou duas décadas depois. Embora a situação tenha melhorado, o risco de extinção ainda existe. Tamanduá-bandeira © joelfotos / Pixabay Este gigante que ama comer formigas — pode devorar até 30 mil por dia, inclusive cupins — é também um forte: com seus 2,20 metros, consegue até brigar com uma onça pintada. E vencer. Embora ele exista originalmente nas três Américas, já foi extinto em países como Estados Unidos e em Estados brasileiros como Espírito Santo e Rio e Janeiro. Onça pintada",381
1,Qual a maior característica da fauna brasileira?,2605802,84.615746,1,clueweb22-pt0000-55-02859_0,"Fauna e Flora: : entenda o que são e quais as diferenças (com exemplos) - Toda Matéria Biologia Fauna e Flora A fauna é o grupo de animais de determinada região. Já a flora é a representação das espécies vegetais. Quando se fala em fauna e flora brasileira, estão sendo referidos a totalidade de animais e plantas que tem seu habitat em um dos biomas existentes no Brasil. No Brasil, existem seis grandes biomas: cerrado, caatinga, mata atlântica, pampas, pantanal e a amazônia. Ou seja, cada comunidade apresenta características específicas e sua própria biodiversidade. Os animais (fauna) e as plantas (flora) variam de acordo com o meio em que se encontram. O que é fauna? A fauna representa a comunidade de espécies animais que habitam um ambiente específico ou alargado. Desse modo, o termo pode ser empregado em relação a um bioma específico, por exemplo, fauna do cerrado ou, em geral, como fauna brasileira ou mundial. Também pode possuir um sentido ainda mais amplo, relativo a um período: por exemplo, \""fauna do período jurássico\"". Em outras palavras, a representação do coletivo de animais de determinado local ou período é chamada fauna.",314
2,Qual a maior característica da fauna brasileira?,7028401,84.480431,1,clueweb22-pt0001-46-05303_17,"http://www.klimanaturali.org/2014/03/biomas-e-fitofisionomias-brasileiros.html 8/17 f15/03/2018 BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA - Klima Naturali™ colonização no Brasil e os ciclos de desenvolvimento do país SITE  BRASIL  MUNDO  CIÊNCIAS  GERAL  levaram o homem a ocupar e destruir parte desse espaço. Fauna e Flora Rica em diversidade de espécies, a Mata Atlântica está entre Pesquisa no Site...  as cinco regiões do mundo com maior número de espécies endêmicas. Os animais podem ser considerados generalistas ou especialistas. Os generalistas apresentam hábitos alimentares variados, alta taxa de crescimento e dispersão; vivem em áreas de vegetação aberta e secundária, tolerantes e capazes de aproveitar diferentes recursos oferecidos pelo meio ambiente. Os animais especialistas são mais exigentes em relação aos habitats nos quais vivem, com dieta especí ca, uma alteração no meio ambiente exige dos animais especialistas a procura de novos habitats. Grupos da Mata Atlântica Mamíferos – A Mata Atlântica possui cerca de 250 espécies de mamíferos, das quais 55 são endêmicas. Os mamíferos são os",347
3,Qual a maior caracter

output_filename=tests/bm25_e5_1M_docs_fixed_000_999_sampled_50_queries_run.txt


In [89]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {bm25_e5_filename}

retrievers_results.append({'retriever':'BM25+E5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [90]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4487
1,BM25+E5,0.6393
2,BM25+mT5,0.7140
3,SPLADEv2_pt-BR,0.5828
4,ColBERT-X_mMARCO_pt-BR,0.6301
5,E5,0.5589
6,E5 base,0.3983
7,SPLADEv2_pt-BR + mT5,0.6999
8,E5 + ColBERT-X mMARCO_pt-BR RRF,0.6408
9,E5 + SPLADEv2_pt-BR RRF,0.6309


## 1M dataset

### Compute BM25 1M massiveweb filtered nDCG@10

In [43]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [44]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADA v2 massiveweb filtered nDCG@10

In [45]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {ADAV2_RUN}

retrievers_1M_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 1M massiveweb filtered nDCG@10

In [46]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_MT5_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25+mT5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + E5 1M massiveweb filtered nDCG@10

In [47]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {bm25_e5_filename}

retrievers_1M_results.append({'retriever':'BM25+E5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [49]:
pd.DataFrame(retrievers_1M_results).sort_values('nDCG@10')

,retriever,nDCG@10
0,BM25 1M massiveweb,0.4168
1,ColBERT-X_mMARCO_pt-BR 1M massiveweb,0.5046
4,BM25+E5 1M massiveweb,0.5634
2,ADA v2 1M massiveweb,0.5825
3,BM25+mT5 1M massiveweb,0.6777


In [92]:
31.5986871 * 1e+7 * 1e-3 * 0.00013

41.07829322999999